In [1]:
import scipy.stats as stats
import os
import numpy as np
import pandas as pd
import random
import re
%matplotlib inline
import statsmodels.formula.api as sm
from pandas.stats.api import ols
import pylab
import matplotlib.pyplot as plt
from sklearn import linear_model
import math

In [2]:
data = pd.read_csv('DB_foranalysis.csv')

In [4]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [6]:
data.head(2)

,Neighborhoods,MSE,x,y,Avg_Familysize,Avg_ENG,Avg_School,Avg_Income
0,sunset-park,0.000713,-73.982874,40.641335,3.864400,3256.000,69.920000,0.706214
1,flushing,0.000223,-73.816388,40.759428,3.383333,3476.375,39.208333,0.593954


In [62]:
#data preperation
data_y = np.asarray(data.iloc[:,1])
data_x = np.asarray(data.iloc[:,4:8])
print data_x.shape
print data_y.shape

(225, 4)
(225,)


In [78]:
#making sure I have the right x and y 
print data.iloc[:,1].head()
print data.iloc[:,4:8].head()

0    0.000713
1    0.000223
2    0.000019
3    0.000046
4    0.000021
Name: MSE, dtype: float64
   Avg_Familysize      Avg_ENG  Avg_School  Avg_Income
0        3.864400  3256.000000   69.920000    0.706214
1        3.383333  3476.375000   39.208333    0.593954
2        3.272222  3796.111111   53.518519    0.563156
3        3.272222  3796.111111   53.518519    0.563156
4        3.285000  3610.500000   51.833333    0.683494


In [64]:
#OLS multivariave model
mod = sm.OLS(data_y,data_x).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.107
Date:                Mon, 18 Apr 2016   Prob (F-statistic):              0.354
Time:                        19:20:13   Log-Likelihood:                -1797.6
No. Observations:                 225   AIC:                             3603.
Df Residuals:                     221   BIC:                             3617.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1           123.3569     81.268      1.518      0.130       -36.803   283.517
x2             0.0349      0.043      0.814      0.416        -0.050     0.119
x3            -2.2031      2.706     -0.814      0.416        -7.536     3.130
x4          -603.4686    394.022     -1.532      0.127     -1379.991   173.053
==============================================================================
Omnibus:                      499.336   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           436855.963
Skew:                          14.590   Prob(JB):                         0.00
Kurtosis:                     216.884   Cond. No.                     3.48e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.48e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [85]:
# train and test data sets divided 80 / 20 % 
# train 80%  and test 20% 

y_test = np.asarray(data.iloc[-45:,1])
y_train = np.asarray(data.iloc[45:,1])

x_test = np.asarray(data.iloc[-45:,4:8])
x_train = np.asarray(data.iloc[45:,4:8])

print 'y test size',y_test.shape
print 'y train size',y_train.shape

print 'x test size',x_test.shape
print 'x train size',x_train.shape

y test size (45,)
y train size (180,)
x test size (45, 4)
x train size (180, 4)


### LASSO

In [86]:
Lasso=linear_model.Lasso(fit_intercept=False, alpha=50,max_iter=100000)
Lasso.fit(x_train,y_train)
p_IS=Lasso.predict(x_train)
err_IS=p_IS-y_train
R_2_IS_Lasso=1-np.var(err_IS)/np.var(y_train)
print("The R-squared we found for IS Lasso(If we set intercept= False) is:\n {0}".format(R_2_IS_Lasso))

The R-squared we found for IS Lasso(If we set intercept= False) is:
 0.00659011062481


In [84]:
Lasso_coef=Lasso.coef_
#Out of sample:
p_OS=Lasso.predict(x_test)
err_OS=p_OS-y_test
R_2_OS_Lasso=1-np.var(err_OS)/np.var(y_test)
print("The R-squared we found for OS Lasso(if we set intercept=False) is:\n {0}".format(R_2_OS_Lasso))

The R-squared we found for OS Lasso(if we set intercept=False) is:
 -9971.74625528


#### lasso coefficients

In [61]:
#lasso on the entire dataset
# alpha= 0 is equivelent to an OLS model 
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=20,fit_intercept=True,max_iter=10000)
res = lasso.fit(data_x,data_y)
res.coef_.tolist()

[0.0, 0.04153466915410821, -2.0379403435387595, -0.0]

In [32]:
# features 2 and 3 are important ? english and school

### OLS

In [69]:
# In the sample. (Only use training set)
result=ols(y=y_train,x=pd.DataFrame(x_train))
#print(result.summary_as_matrix.T)
R_2_IS=result.r2 # get R2
OLS_coef=result.beta
#Out of sample
a=np.array(x_test) #makes sure conver pd data to np array
b=np.array(result.beta) #makes sure conver pd data to np array
print('OLS regression coefficients={0}'.format(b))
c=np.sum(a*b[0:-1],axis=1)+b[-1] #b is estimated coefficients, a is
#prediction data, b[-1] is intercept. This is for predicted y
error=y_test-c # y_predict is real value, c is the value we guessed
R_2_OS=1-error.var()/y_test.var() # this is out of sample R2
print("The R-squared we found for in-sample (IS) OLS is: {0}".format(R_2_IS))
print("The R-squared we found for out-of-sample (OS) OLS is: {0}".format(R_2_OS))

OLS regression coefficients=[  4.64306599e+02   1.16061322e-01  -5.56246426e+00  -7.48981321e+02
  -1.10380038e+03]
The R-squared we found for in-sample (IS) OLS is: 0.0289045655418
The R-squared we found for out-of-sample (OS) OLS is: -30012.4892966


In [70]:
### r-squared off sample is WAY bellow zero - we have an overfitting issue

### RIDGE

In [92]:
Ridge=linear_model.Ridge(fit_intercept=True,alpha=1) #try Ridge with an arbitrary regularization parameter lambda=1
Ridge.fit(x_train,y_train)
#In the sample:
p_IS=Ridge.predict(x_train)
err_IS=p_IS-y_train
R_2_IS_Ridge=1-np.var(err_IS)/np.var(y_train)
print("The R-squared we found for IS Ridge is: {0}".format(R_2_IS_Ridge))

The R-squared we found for IS Ridge is: 0.0276461785143


In [93]:
Ridge_coef=Ridge.coef_
#Out of sample
p_OS=Ridge.predict(x_test)
err_OS=p_OS-y_test
R_2_OS_Ridge=1-np.var(err_OS)/np.var(y_test)
print("The R-squared we found for OS Ridge is: {0}".format(R_2_OS_Ridge))

The R-squared we found for OS Ridge is: -21030.460784
